In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
!pip install optuna
import optuna.integration.lightgbm as lgb

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import os
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# N-Lags model check
# !pip install yfinance
import yfinance as yf
# 他ファイルのインポート・リロード------------------------
import imp
import joblib

def data_process(data):
    count = 0
    preprocess_data = pd.DataFrame()

    # # 作業ディレクトリの移動と確認
    os.chdir('/Users/suzukiryotaro/Desktop/python_stock/machine_learning/data_process')
    # 他ファイルのインポート・リロード
    import data_process
    imp.reload(data_process)
    for  file in data['symbol'] :
        count = count +1
        print(count)
        print(file)
        try:
            df = yf.download(file, start='2014-01-01')
            df = data_process.create_data(df)
            df = df['2016-01-06':'2020-12-31']
            preprocess_data = preprocess_data.append(df)

        except: 
            print('errror')
            continue
    # drop row contains NaN
    save_path = '/Users/suzukiryotaro/Desktop/python_stock/machine_learning/ansamble/lightgbm/optuna/preprocss_data.csv'
    preprocess_data.dropna(inplace=True)
    preprocess_data.to_csv(save_path)
    print('saved')

    return preprocess_data


#全時系列データの呼び出し
model_stock = 'sandp500'
path = '/Users/suzukiryotaro/Desktop/python_stock/stock_data/stock_code/sandp500.csv'
data = pd.read_csv(path)
print(data)

#データの前処理　※一度やったら飛ばす

# df = data_process(data)



model_name = 'lightgbm_0708_optuna'


df = pd.read_csv(save_path, index_col=0)
print(df)
#選択する特徴量
cols =['return','sma210'

          ,'ratio_sma75_30', 'ratio_sma75_150', 'ratio_sma105_30', 
          'ratio_sma105_150', 'ratio_sma135_150'
          ,'Highest161','adosc', 'sma10', 
        'adosc-SG' ,'typical-price',"ema270"
      ,'Highest121','Highest45,81days_ago',

'today_by_sma30ratio','today_by_sma45ratio','today_by_sma60ratio','today_by_sma120ratio'
,'Adjclose_today_20daysago_ratio','Adjclose_today_15daysago_ratio','Adjclose_today_10daysago_ratio','Adjclose_today_5daysago_ratio',


          ]


X = df[cols].copy()
import re
X = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
y = X.pop('return')
print(X)
print(y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=11)
# train用・valid用のデータセットを生成する
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val) 

params = {'metric': 'rmse',
          'max_depth' : 20,
          'num_leaves': 23, # 決定木の複雑度を調整（初期値31）'num_leaves': 23, # 決定木の複雑度を調整（初期値31）
          'boosting_type': 'gbdt', # 勾配ブースティング
          'objective': 'regression', # 目的 : 回帰  
#           'num_leaves': 23, # 決定木の複雑度を調整（初期値31）
          'min_data_in_leaf': 15, # データの最小数（初期値20） 
          "learning_rate": 0.025400643228660445,
          'gamma':0.20387750446308275, 
          }

# model = lgb.train(params,
#                 lgb_train,
#                 valid_sets=lgb_eval,
#                 num_boost_round=10000,
#                 early_stopping_rounds=100,
#                 verbose_eval=50
#                 )

best_params, tuning_history = dict(), list()
booster = lgb.train(params, 
                    lgb_train,
                    valid_sets=lgb_eval,
                    num_boost_round=10000,
                    early_stopping_rounds=100,
                    verbose_eval=200,
                    )




joblib.dump(model, '/Users/suzukiryotaro/Desktop/python_stock/machine_learning/ansamble/lightgbm/optuna/optuna_lightgbm.ipynb/{}.joblib'.format(model_name)) 

best_params = model.params
best_params

# 特徴量重要度の算出 (データフレームで取得)
# .drop('return',axis=1)
cols = list(X.columns)         # 特徴量名のリスト(目的変数CRIM以外)
f_importance = np.array(model.feature_importance()) # 特徴量重要度の算出
f_importance = f_importance / np.sum(f_importance)  # 正規化(必要ない場合はコメントアウト)
df_importance = pd.DataFrame({'feature':cols, 'importance':f_importance})
df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート
display(df_importance)



# 特徴量重要度を棒グラフでプロットする関数 
def plot_feature_importance(df): 
    n_features = len(df)                              # 特徴量数(説明変数の個数) 
    df_plot = df.sort_values('importance')            # df_importanceをプロット用に特徴量重要度を昇順ソート 
    f_importance_plot = df_plot['importance'].values  # 特徴量重要度の取得 
    plt.barh(range(n_features), f_importance_plot, align='center') 
    cols_plot = df_plot['feature'].values             # 特徴量の取得 
    plt.yticks(np.arange(n_features), cols_plot)      # x軸,y軸の値の設定
    plt.xlabel('Feature importance')                  # x軸のタイトル
    plt.ylabel('Feature')                             # y軸のタイトル
# 特徴量重要度の可視化
plot_feature_importance(df_importance)




# model = joblib.load("/content/drive/MyDrive/machine_learning/light_gbm/model/{}.joblib".format(model_name))


# # # 作業ディレクトリの移動と確認
# os.chdir('/content/drive/MyDrive/machine_learning/light_gbm/backtest/')
# # 他ファイルのインポート・リロード
# import backtest_lightgbm
# imp.reload(backtest_lightgbm)


# model_name_list = [model_name]
# model_list = [model]
# backtest_lightgbm.backtest(model_list, model_name_list,model,cols)


    symbol                 Name                  Sector
0      MMM           3M Company             Industrials
1      AOS      A.O. Smith Corp             Industrials
2      ABT  Abbott Laboratories             Health Care
3     ABBV          AbbVie Inc.             Health Care
4     ABMD              Abiomed             Health Care
..     ...                  ...                     ...
500    YUM      Yum! Brands Inc  Consumer Discretionary
501   ZBRA   Zebra Technologies  Information Technology
502    ZBH        Zimmer Biomet             Health Care
503   ZION        Zions Bancorp              Financials
504    ZTS               Zoetis             Health Care

[505 rows x 3 columns]
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2016-01-06  145.589996  145.759995  143.419998  144.490005  122.994637   
2016-01-07  142.520004  143.130005  140.630005  140.970001  119.998299   
2016-01-

[I 2021-07-09 11:25:34,996] A new study created in memory with name: no-name-7b74419a-6c6d-40ec-867c-9c7baef2b094



  0%|          | 0/7 [00:00<?, ?it/s]


feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.021665 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5358
[LightGBM] [Info] Number of data points in the train set: 434105, number of used features: 22
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Info] Start training from score 0.027948
Training until validation scores don't improve for 100 rounds
[200]	valid_0's rmse: 0.451657
[400]	valid_0's rmse: 0.446728
[600]	valid_0's rmse: 0.44




feature_fraction, val_score: 0.406306:   0%|          | 0/7 [01:25<?, ?it/s]


feature_fraction, val_score: 0.406306:  14%|#4        | 1/7 [01:25<08:30, 85.03s/it][I 2021-07-09 11:27:00,052] Trial 0 finished with value: 0.40630605280755844 and parameters: {'feature_fraction': 1.0}. Best is trial 0 with value: 0.40630605280755844.



feature_fraction, val_score: 0.406306:  14%|#4        | 1/7 [01:25<08:30, 85.03s/it]

[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003429 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5358
[LightGBM] [Info] Number of data points in the train set: 434105, number of used features: 22
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Info] Start training from score 0.027948
Training until validation scores don't improve for 100 rounds
[200]	valid_0's rmse: 0.

...




feature_fraction, val_score: 0.406306:  71%|#######1  | 5/7 [08:39<02:48, 84.49s/it]


feature_fraction, val_score: 0.406306:  86%|########5 | 6/7 [08:39<01:29, 89.63s/it][I 2021-07-09 11:34:14,714] Trial 5 finished with value: 0.40742368510345667 and parameters: {'feature_fraction': 0.6}. Best is trial 0 with value: 0.40630605280755844.



feature_fraction, val_score: 0.406306:  86%|########5 | 6/7 [08:39<01:29, 89.63s/it]

[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005101 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5358
[LightGBM] [Info] Number of data points in the train set: 434105, number of used features: 22
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Info] Start training from score 0.027948
Training until validation scores don't improve for 100 rounds
[200]	valid_0's rmse: 0.




feature_fraction, val_score: 0.406306:  86%|########5 | 6/7 [10:09<01:29, 89.63s/it]


feature_fraction, val_score: 0.406306: 100%|##########| 7/7 [10:09<00:00, 87.12s/it]



  0%|          | 0/20 [00:00<?, ?it/s]


num_leaves, val_score: 0.406306:   0%|          | 0/20 [00:00<?, ?it/s][W 2021-07-09 11:35:45,016] Trial 7 failed because of the following error: LightGBMError('Check failed: (num_leaves) <= (131072) at /tmp/pip-req-build-5153luwz/compile/src/io/config_auto.cpp, line 325 .\n')
Traceback (most recent call last):
  File "/opt/anaconda3/envs/py37/lib/python3.7/site-packages/optuna/_optimize.py", line 216, in _run_trial
    value_or_values = func(trial)
  File "/opt/anaconda3/envs/py37/lib/python3.7/site-packages/optuna/integration/_lightgbm_tuner/optimize.py", line 249, in __call__
    booster = lgb.train(self.lgbm_params, train_set, **kwargs)
  File "/opt/anaconda3/envs/py37/lib/python3.7/site-packages/lightgbm/engine.py", line 228, in train
    booster = Booster(params=pa

[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15


LightGBMError: Check failed: (num_leaves) <= (131072) at /tmp/pip-req-build-5153luwz/compile/src/io/config_auto.cpp, line 325 .
